In [ ]:
# Bayesian Optimization with Random Forest

In [7]:
!pip install bayesian-optimization scikit-learn pandas

In [8]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score, train_test_split
from bayes_opt import BayesianOptimization

In [9]:
data = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv', sep=';')
X = data.drop('quality', axis=1)
y = data['quality']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [10]:
def objective(n_estimators, max_depth, min_samples_split, max_features):
    model = RandomForestRegressor(n_estimators=int(n_estimators),
                                  max_depth=int(max_depth),
                                  min_samples_split=int(min_samples_split),
                                  max_features=min(max_features, 0.999),  # Fraction, must be <= 1.0
                                  random_state=42)

    return -1.0 * cross_val_score(model, X_train, y_train, cv=3, scoring="neg_mean_squared_error").mean()

In [11]:
# Bounds for hyperparameters
param_bounds = {
    'n_estimators': (10, 250),
    'max_depth': (1, 50),
    'min_samples_split': (2, 25),
    'max_features': (0.1, 0.999),
}

In [12]:
optimizer = BayesianOptimization(f=objective, pbounds=param_bounds, random_state=42)
optimizer.maximize(init_points=5, n_iter=15)

|   iter    |  target   | max_depth | max_fe... | min_sa... | n_esti... |
-------------------------------------------------------------------------
| 1         | 0.3948    | 19.35     | 0.9547    | 18.84     | 153.7     |
| 2         | 0.3985    | 8.645     | 0.2402    | 3.336     | 217.9     |
| 3         | 0.3797    | 30.45     | 0.7366    | 2.473     | 242.8     |
| 4         | 0.3808    | 41.79     | 0.2909    | 6.182     | 54.02     |
| 5         | 0.3886    | 15.91     | 0.5718    | 11.93     | 79.89     |
| 6         | 0.3925    | 8.632     | 0.3537    | 4.404     | 217.6     |
| 7         | 0.4288    | 7.871     | 0.1       | 2.103     | 218.3     |
| 8         | 0.4288    | 7.204     | 0.1       | 2.0       | 218.8     |
| 9         | 0.4389    | 6.551     | 0.1       | 2.0       | 217.2     |
| 10        | 0.469     | 4.289     | 0.1       | 2.0       | 216.5     |
| 11        | 0.4989    | 3.357     | 0.1       | 2.0       | 214.5     |
| 12        | 0.5087    | 1.0       | 

In [13]:
best_params = optimizer.max['params']
best_params

{'max_depth': 1.0,
 'max_features': 0.1,
 'min_samples_split': 2.0,
 'n_estimators': 210.79678776201987}

In [14]:
final_model = RandomForestRegressor(n_estimators=int(best_params['n_estimators']),
                                   max_depth=int(best_params['max_depth']),
                                   min_samples_split=int(best_params['min_samples_split']),
                                   max_features=best_params['max_features'],
                                   random_state=42)
final_model.fit(X_train, y_train)
score = final_model.score(X_test, y_test)
print(f"Test R^2 Score: {score}")

Test R^2 Score: 0.11323487205843263


In [16]:
best_params_formatted = {
    'n_estimators': int(best_params['n_estimators']),
    'max_depth': int(best_params['max_depth']),
    'min_samples_split': int(best_params['min_samples_split']),
    'max_features': best_params['max_features']
}
best_params_formatted

{'n_estimators': 210,
 'max_depth': 1,
 'min_samples_split': 2,
 'max_features': 0.1}

In [17]:
optimized_rf = RandomForestRegressor(**best_params_formatted, random_state=42)

In [18]:
optimized_rf.fit(X_train, y_train)

RandomForestRegressor(max_depth=1, max_features=0.1, n_estimators=210,
                      random_state=42)

In [19]:
score = optimized_rf.score(X_test, y_test)
print(f"Test R^2 Score with Optimized Hyperparameters: {score}")

Test R^2 Score with Optimized Hyperparameters: 0.11323487205843263
